In [13]:
import cPickle
import gzip
import os
import sys
import timeit
import numpy
import theano
import theano.tensor as T
import numpy as np

In [14]:
'''download the mnist data set no check for path or for if already downloaded'''
def load_data(dataset): 
    #include checking for if it already exists
    origin = ( 'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz' )
    dataset = 'MNIST'
    import urllib
    urllib.urlretrieve(origin, dataset)
    '''downloads the file to the current working directoty and makes a file in that one'''
    f = gzip.open(dataset, 'rb')#looks in current directy on path
    train_set, test_set, val_set = cPickle.load(f)
    f.close()
    def shared_dataset(data_xy, borrow = True):
        data_x, data_y = data_xy 
        shared_x = theano.shared(numpy.asarray(data_x, dtype = theano.config.floatX), 
                                 borrow = borrow )
        shared_y = theano.shared(numpy.asarray(data_y, dtype = theano.config.floatX), 
                                 borrow = borrow )
        return shared_x, T.cast(shared_y, 'int32' )
    test_set_x, test_set_y = shared_dataset(test_set)
    valid_set_x, valid_set_y = shared_dataset(val_set)
    train_set_x, train_set_y = shared_dataset(train_set)
    rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y),
            (test_set_x, test_set_y)]
    return rval 


In [41]:
class LogisticRegression(object):
    import numpy as np
    def __init__(self, input, n_in, n_out):
        '''initialize with zeroes a matrix of ninxnout'''
        self.W = theano.shared( value = numpy.zeros((n_in,n_out), dtype = theano.config.floatX), 
                               name = 'W', borrow = True)
        self.b = theano.shared(value = numpy.zeros((n_out,), dtype = theano.config.floatX), 
                               name = 'b', borrow = True)
        self.p_y_given_x = T.nnet.softmax(T.dot(input,self.W) + self.b)
        self.y_pred = T.argmax(self.p_y_given_x, axis = 1)
        self.params = [self.W, self.b]
        self.input = input
    def negative_log_likelihood(self,y):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y])
    def errors(self,y):
        if y.ndim != self.y_pred.ndim: 
            raise TypeError('y should have the same shape as self.y_pred'('y', y.type, 'yprred',
                                                                          self.y_pred.type))
        if y.dtype.startswith('int'):
            return T.mean(T.neq(self.y_pred, y))
        else:
            raise NotImplementedError()
            

In [6]:
train ,valid, test = load_data('MNIST')
tr_x, tr_y = train[0] , train[1]
val_x, val_y = valid[0] , valid[1]
test_x, test_y = test[0] , test[1]

n_train_batches = tr_x.
n_val_batches = 
n_test_batches = 


Shape.0

In [9]:





x = T.matrix('x')
y = T.ivector('y')
classifier = LogisticRegression(x, 28*28, 10) # this is defining the function that's going to be ushelped
classifier.negative_log_likelihood(y)

Elemwise{neg,no_inplace}.0

In [1]:
def data(dataset,batch_size):
    datasets=dataset
    train_set_x , train_set_y = datasets[0]
    valid_set_x, valid_set_y = datasets[1]
    test_set_x, test_set_y = datasets[2]
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
    n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size
    return train_set_x , train_set_y, valid_set_x, valid_set_y, test_set_x, test_set_y, n_train_batches, n_valid_batches,n_test_batches
ds = load_data('MNIST')
train_set_x , train_set_y, valid_set_x, valid_set_y, test_set_x, test_set_y, n_train_batches, n_valid_batches,n_test_batches = data(ds , 600)





NameError: name 'load_data' is not defined

In [55]:
def theanofunctions(batch_size,datax,datay):
    learning_rate = 0.01; #hard coded change latter
    index = T.lscalar()
    x = T.matrix('x')
    y= T.ivector('y')
    classifier = LogisticRegression(input = x, n_in = 28*28, n_out = 10)
    cost = classifier.negative_log_likelihood(y)
    grad_W = T.grad(cost = cost, wrt = classifier.W)
    grad_b = T.grad(cost = cost, wrt = classifier.b)
    updates = [(classifier.W, classifier.W  - grad_W*learning_rate) , (classifier.b, classifier.b  - grad_b*learning_rate)] 
    def createmodel(outputs ,index, datax, datay, batch_size,updates):
        model = theano.function(inputs = [index] , outputs = outputs , updates = updates
                                , givens = {x : datax[index*batch_size : (index+1) * batch_size], 
                                                            y : datay[index*batch_size : (index+1) * batch_size]})
        return model
    
    test_model = createmodel(classifier.errors(y), index, test_set_x, test_set_y, 600, None)
    valid_model = createmodel(classifier.errors(y) , index, valid_set_x, valid_set_y, 600,None) 
    train_model = createmodel(cost, index, train_set_x, train_set_y, 600, updates)
    return (test_model, valid_model, train_model)

In [62]:
test_model, valid_model, train_model = theanofunctions(600,train_set_x , train_set_y)

In [ ]:
def trainingthemodel(n_train_batches, n_epochs, test_model, valid_model, train_model):
    #paramaters that are hard coded
    patience = 1 , patience_increase = 2
    improvement_threshold = 0.995 
    validation_frequency = 1

    best_validation_loss = 1000000000
    test_score = 0
    done_looping = False
    epoch =0
    while ( epoch < n_epochs ) and (not done_looping):
        epoch = epoch + 1
        for mini_index in xrange(n_train_batches):
            print('check')
            avg_cost = train_model(mini_index)
            #number of iteration
            iter= (epoch -1)*n_train_batches + mini_index
            #check with the validation set and see if there is an improvment
            if (iter + 1 )% validation_frequency == 0:
                validation_losses = [valid_model(i) for i in xrange(n_train_batches) ]
                this_val_loss = np.mean(validation_losses)
                #check validation loss
                print('check')
                print(this_val_loss)
                if this_val_loss < best_validation_loss:
                    #check if better than the current best loss increase patience
                    
                    print('check')
                    best_validation_loss = this_val_loss
                    test_losses = [test_model(i) for i in xrange(n_test_batches)]
                    test_score = np.mean(test_losses)
                    #dumpt model 
                    with open('best_model.pkl', 'w') as f:
                        cPickle.dump(classifier, f)
            if patience <= iter:
                done_looping = True
                break


In [91]:
def predict():
    classifier = cPickle.load(open('best_model.pkl'))
    predict_model = theano.function(inputs = [classifier.input] ,
                                  outputs = classifier.y_pred)
    dataset = 'mnist.pkl.gz'
    datasets = load_data(dataset)
    test_set_x = test_set_y = datasets[2]
    test_set_x = test_set_x.get_value()
    predicted_values = predict_model(test_set_x[:10])
    print ("Predicted values for the first 10 examples in test set:")
    print predicted_values
if __name__ =='__main__':
    trainingthemodel(600,10,test_model, valid_model, train_model )  
    #predict()

check
check
nan
check
check
nan


In [74]:
predict()

IOError: [Errno 2] No such file or directory: 'best_model.pkl'

In [9]:
def sgd_optimization_mnist(learning_rate = 0.13, n_epochs = 1000, dataset = 'MNIST', batch_size = 600):
    ##create 3 data sets 
    datasets = load_data(dataset)
    train_set_x , train_set_y = datasets[0]
    valid_set_x, valid_set_y = datasets[1]
    test_set_x, test_set_y = datasets[2]
    
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
    n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size
    ###one cut off for a function 
    #build actual model 
    print ('building the model ')
    index = T.lscalr() #index to a minibatch
    x = T.matrix('x')
    y = T.ivector('y')
    
    classifier = LogisticRegression(input = x, n_in = 28*28, n_out = 10)
    cost = classifier.negative_log_likelihood(y)
    test_model = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: test_set_x[index * batch_size: (index + 1) * batch_size],
            y: test_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    validate_model = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: valid_set_x[index * batch_size: (index + 1) * batch_size],
            y: valid_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )
    g_w = T.grad(cost = cost, wrt = classifier.W)
    g_b = T.grad(cost = cost, wrt= classifier.b)
    
    updates = [(classifier.W, classifier.W - learning_rate * g_W),
               (classifier.b, classifier.b - learning_rate * g_b)]
    train_model = theano.function( inputs = [index], outputs = cost, updates = updates, 
                                  givens = {
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            y: train_set_y[index * batch_size: (index + 1) * batch_size]
        }
                                  )
    ##training the model
    print('training model')
    patience = 5000
    patience_increase = 2
    improvement_threshold = 0.995 
    validation_frequency = min(n_train_batches, patience/2)

    best_validation_loss = numpy.inf
    test_score = 0
    start_time_ =timit.default.timer()
    done_looping = False
    epoch =0
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1 
        for minibatch_index in xrange(n_train_batches):
            minibatch_avg_cost = train_model(minibatch_index)
            iter = (epoch -1)*n_train_batches + minibatch_index
            if (iter + 1 )% validation_frequency == 0:
                validation_losses = [validate_model(i) for i in xrange(n_valid_batches)]
                this_validation_loss = numpy.mean(validation_losses)
                print (
                        'epoch %i, minibatch %i/%i, validation error %f %%' %
                        (
                            epoch,
                            minibatch_index + 1,
                            n_train_batches,
                            this_validation_loss * 100.
                        )
                    )
                if this_validation_loss < best_validation_loss * improvment_threshold: 
                    patience = max(patience, iter*patience_increase)

                    best_validation_loss = this_validation_loss
                    test_losses = [test_model(i) for i in xrange(n_test_batches)]
                    test_score = numpy.mean(test_losses)

                    print(
                            (
                                '     epoch %i, minibatch %i/%i, test error of'
                                ' best model %f %%'
                            ) %
                            (
                                epoch,
                                minibatch_index + 1,
                                n_train_batches,
                                test_score * 100.
                            )
                        )

                        # save the best model
                    with open('best_model.pkl', 'w') as f:
                            cPickle.dump(classifier, f)

                if patience <= iter:
                    done_looping = True
                    break

        end_time = timeit.default_timer()
        print(
            (
                'Optimization complete with best validation score of %f %%,'
                'with test performance %f %%'
            )
            % (best_validation_loss * 100., test_score * 100.)
        )
        print 'The code run for %d epochs, with %f epochs/sec' % (
            epoch, 1. * epoch / (end_time - start_time))
        print >> sys.stderr, ('The code for file ' +
                              os.path.split(__file__)[1] +
                              ' ran for %.1fs' % ((end_time - start_time)))


In [18]:
predict():
'''examples of how to load a trained model and use it to predict labels'''
    classifier = cPickle.load(open('best_model.pkl'))
    predict_model = theano.function(inputs = [classifier.input]. outputs = classifier.y_pred)
    dataset = 'mnist.pkl.gz'
    datasets = load_data(dataset)
    test_set_x , test_set_y = datasets[2]
    test_set_x = test_set_x.get_value()
    predicted_values = predict_model(test_set_x[:10])
    print(predicteed_values)
if __name__ == "__main__"
    sgd_optimization_mnist()

((array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32),
  array([5, 0, 4, ..., 8, 4, 8])),
 (array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32),
  array([3, 8, 6, ..., 5, 6, 8])),
 (array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32),
  arra

In [20]:
os.chdir('home/aplstudent/MNIST')

OSError: [Errno 2] No such file or directory: 'home/aplstudent/MNIST'

SyntaxError: invalid syntax (<ipython-input-1-9cae6bd2726c>, line 1)

In [23]:
os.getcwd()

'/home/aplstudent'

In [0]:
'''download the mnist data set no check for path or for if already downloaded'''
def load_data(dataset): 
    #include checking for if it already exists
    origin = ( 'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz' )
    dataset = 'MNIST'
    import urllib
    urllib.urlretrieve(origin, dataset)
    '''downloads the file to the current working directoty and makes a file in that one'''
    f = gzip.open(dataset, 'rb')#looks in current directy on path
    train_set, test_set, val_set = cPickle.load(f)
    f.close()
    def shared_dataset(data_xy, borrow = True):
        data_x, data_y = data_xy 
        shared_x = theano.shared(numpy.asarray(data_x, dtype = theano.config.floatX), 
                                 borrow = borrow )
        shared_y = theano.shared(numpy.asarray(data_y, dtype = theano.config.floatX), 
                                 borrow = borrow )
        return shared_x, T.cast(shared_y, 'int32' )
    test_set_x, test_set_y = shared_dataset(test_set)
    valid_set_x, valid_set_y = shared_dataset(val_set)
    train_set_x, train_set_y = shared_dataset(train_set)
    rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y),
            (test_set_x, test_set_y)]
    return rval 

In [7]:

(tr_x , tr_y ) , (valid_x , valid_y) , (test_x, test_y) = load_data('MNIST')

In [3]:
import theano
import numpy as np
import pandas as pd
from theano import tensor as T

In [4]:
# load data
#declare variables 

def init_weights(shape):
    return theano.shared(np.asarray(np.random.rand(*shape), dtype = theano.config.floatX))
def model(X,w):
    return T.nnet.softmax(T.dot(X,w))
X = T.fmatrix()
Y = T.ivector()
W = init_weights((784,10))
p_y_given_x= model(X,W)
y_pred= T.argmax(p_y_given_x)

#cost = T.mean(T.nnet.categorical_crossentropy(py_x, Y))
cost = T.mean(T.log(p_y_given_x)[T.arange(Y.shape[0]) , Y])
gradient = T.grad(cost , wrt = W)
updates = [[W , W - gradient*0.01]]
#compile model and wrtie theano function
train = theano.function ([X,Y], outputs = cost , updates = updates , allow_input_downcast = True )
#do loop to train the function with the data

In [42]:
#loads the dataset from the mnist gzip that's pickled in aplstuent
f = gzip.open('MNIST', 'rb')#looks in current directy on path
train_set, test_set, val_set = cPickle.load(f)
f.close()
tr = one_hot(train_set[0] , 10)
tr_label = one_hot(train_set[1], 10)
o_h = pd.get_dummies(pd.DataFrame(train_set[0]))
pd.concat((pd.DataFrame(train_set[1]) , o_h) ,axis = 1 , join = 'inner').as_matrix()

array([[ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 8.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 8.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [ ]:
#do the training 
for i in range(1000):
    for start , end in zip(range(0, 50000, 128), range(128, 50000, 128)):
        cost = train(o_h[start:end] , train_set[1][start:end])
        print(cost)

-1822.20024386
-1610.48979376
-1682.0762309
-1638.58364126
-1711.75903268
-1806.83840085
-1752.1267742
-1696.33847357
-1542.47798884
-1916.788487
-2143.7889713
-1664.51811265
-1572.21142383
-1677.21739262
-1725.26379068
-1869.20944373
-1717.08864435
-1602.29449522
-1707.24658228
-1783.44205794
-1740.24376108
-1629.13368618
-1711.55958389
-1637.34457677
-1670.96497286
-1666.76196622
-1619.31152192
-1651.4356951
-1700.42013537
-1624.76948189
-1716.08267368
-1745.07064187
-1728.06454976
-1678.16075667
-1565.6749383
-1765.75905079
-1837.46587771
-1616.29090617
-1652.35992871
-1774.34899085
-1904.3421095
-1881.24233322
-1870.5922271
-1869.9577275
-1781.77178749
-1690.99894439
-1875.29159569
-1999.741816
-1927.28737419
-1953.20690155
-1635.83292819
-1701.54667453
-1493.15805723
-1554.49596651
-1965.91712268
-1552.67000205
-1583.4195542
-1389.59778057
-1578.35756743
-1457.19986784
-1755.3335235
-1887.48761144
-1867.52090326
-1674.09418684
-1765.31028604
-1823.64964928
-2088.71851028
-1962.114

In [6]:

import numpy as np
import os

datasets_dir = '/home/aplstudent/'

def one_hot(x,n):
    if type(x) == list:
        x = np.array(x)
    x = x.flatten()
    o_h = np.zeros((len(x),n))
    o_h[np.arange(len(x)),x] = 1
    return o_h

def mnist(ntrain=60000,ntest=10000,onehot=True):
    data_dir = os.path.join(datasets_dir,'mnist/')
    fd = open(os.path.join(data_dir,'train-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd,dtype=np.uint8)
    trX = loaded[16:].reshape((60000,28*28)).astype(float)

    fd = open(os.path.join(data_dir,'train-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd,dtype=np.uint8)
    trY = loaded[8:].reshape((60000))

    fd = open(os.path.join(data_dir,'t10k-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd,dtype=np.uint8)
    teX = loaded[16:].reshape((10000,28*28)).astype(float)

    fd = open(os.path.join(data_dir,'t10k-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd,dtype=np.uint8)
    teY = loaded[8:].reshape((10000))

    trX = trX/255.
    teX = teX/255.

    trX = trX[:ntrain]
    trY = trY[:ntrain]

    teX = teX[:ntest]
    teY = teY[:ntest]

    if onehot:
        trY = one_hot(trY, 10)
        teY = one_hot(teY, 10)
    else:
        trY = np.asarray(trY)
        teY = np.asarray(teY)

        return trX,teX,trY,teY

In [17]:
def one_hot(x,n):
    if type(x) == list:
        x = np.array(x)
    x = x.flatten()
    o_h = np.zeros((x.shape[0],n))
    #o_h[np.arange(x.shape[0],x)] = 1
    return o_h

NameError: name 'o_h' is not defined

In [25]:
#loads the dataset from the mnist gzip that's pickled in aplstuent
f = gzip.open('MNIST', 'rb')#looks in current directy on path
train_set, test_set, val_set = cPickle.load(f)
f.close()
tr = one_hot(train_set[0] , 10)
tr_label = one_hot(train_set[1], 10)
o_h = pd.get_dummies(pd.DataFrame(train_set[0][1:5]))
pd.concat((pd.DataFrame(train_set[1][1:5]) , o_h) ,axis = 1 , join = 'inner')

,0,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
data = pd.DataFrame(train_set[0])#this is the number of classes from 1 to 10
o_h

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
data.iloc[0:10]

numpy.ndarray

In [128]:
tr_label.shape

(50000, 10)

In [108]:
x.flatten().shape[0]

39200000

In [115]:
train_set[0].shape[0]

50000

In [9]:
[a =1,b=3]

SyntaxError: invalid syntax (<ipython-input-9-af9e778d54ec>, line 1)